## Classes and Interfaces
### Item 37: Compose Classes Instead of Nesting Many Levels of Built-in Types
- Avoid making dictionaries with values that are dictionaries, long tuples, or complex nesting of other built-in types.
- Use `namedtuple` for lightweight, immutable data containers before you need the flexibility of a full class.
- Move your bookkeeping code to using multiple classes when your internal state dictionaries get complicated.
#### Example
We want to record the grades of a set of students whose names aren't known in advance. A class can hold the names in a dictionary instead of using a predefined attribute for each student.

In [2]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)
    
book = SimpleGradebook()
book.add_student("Sarah Brook")
book.report_grade("Sarah Brook", 90)
book.report_grade("Sarah Brook", 85)
book.report_grade("Sarah Brook", 95)
print(book.average_grade("Sarah Brook"))


90.0


- Dictionaries and their related built-in types are so easy to use that they have the risk of overextending them to write brittle code. 
- If we want to extend the `SimpleGradebook` class to keep a list of grades by subject, not just overall. This can be done by changing the `_grades` dictionary to hold yet another dictionary that maps the subject name to the grades. 
- The inner dictionary can be a `defaultdict` instance to handle missing subjects.

In [3]:
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {} # Outer dict
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # Inner dict

This seems straightforward, the `report_grade` and `average_grade` methods gain a bit of complexity to deal with the multilevel dictionary. If the requirements change again and we also need to track the weight of each score toward the overall grade in the class, it gets even more complex.
We can make the grades be a tuple and map the subject in the inner dict to a tuple of `(score, weight)`. 

In [4]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))


Now the `average_grade` method now has to loop within a loop and is difficult to read:

In [6]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))

    def average_grade(self, name):
        by_subject = self._grades[name]

        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
        return score_sum / score_count

book = WeightedGradebook()
book.add_student("Sarah Brook")
book.report_grade("Sarah Brook", "Math", 75, 0.05)
book.report_grade("Sarah Brook", "Math", 65, 0.15)
book.report_grade("Sarah Brook", "Math", 70, 0.8)
book.report_grade("Sarah Brook", "Gym", 100, 0.4)
book.report_grade("Sarah Brook", "Gym", 85, 0.6)

print(book.average_grade("Sarah Brook"))


80.25


#### Refactoring to Classes
We can start moving to classes at the bottom of the dependency tree: a single grade. A class seems too heavyweight for such a simple information. A `tuple` though, seems appropriate because grades are immutable. The problem with that is that `tuple` instances are positional. For example, if we want to associate more information with grade than its weight, such as a set of notes from the teacher, we need to rewrite every usage of the two-tuple to be aware that there are now three items present instead of two. 

As soon as you find yourself going longer than a two-tuple, it's time to consider another approach. The `namedtuple` type in the `collections` built-in module does that: It lets you easily define a tiny, immutable data class:

In [ ]:
from collection import namedtuple

Grade = namedtuple("Grade", ("score", "weight"))

The fields are accessible with named attributes. Having named attributes makes it easy to move from a `namedtuple` to a class later if the requirements change again and we need support mutability or behaviours in the simple data containers for example.

#### Limitations of `namedtuple`